|col1|col2|col3|
|:---|:---|:---|
| 1  | ?  | ?  |
| 5  | ?  | ?  |
| 7  |

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras as ks
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Loading Dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
X_train.shape

In [ ]:
np.unique(y_train)

In [ ]:
# Visualization function to plot a sample
def plot_sample(index):
    plt.figure(figsize=(10, 1))  # Define the figure size
    plt.imshow(X_train[index])  # Show the image
    plt.show()

# Plot a sample image
plot_sample(0)  

In [ ]:
plot_sample(1)

In [ ]:
plot_sample(2)

In [ ]:
classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
plt.figure(figsize=(3, 1))
for i in range(5):
    plt.imshow(X_train[i])
    plt.xlabel(classes[y_train[i]]);

In [ ]:
classes[y_train[1]]

## Scaling the dataset

In [ ]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

### One Hot Encoding

In [ ]:
y_train_categorical = ks.utils.to_categorical(y_train, num_classes=10)

In [ ]:
y_train_categorical[:5]

In [ ]:
y_test_categorical = ks.utils.to_categorical(y_test, num_classes=10)

In [ ]:
model = ks.Sequential([
    ks.layers.Flatten(input_shape = (28, 28)),  # this is the input layer
    ks.layers.Dense(500, activation = 'relu'),  # this is the hidden layer and relu is the best option
    ks.layers.Dense(10, activation = 'sigmoid')  # this is the output layer, sigmoid is good with classification
])

# Model compilation
model.compile(optimizer = 'adam',
             loss = 'categorical_crossentropy',  # sparse_categorical_entropy is not used because y_train has been coverted to one hot encoded
             metrics = ['accuracy'])

model.fit(X_train_scaled, y_train_categorical, epochs = 10)

In [ ]:
# Function for 5 hidden layers model and performance test
def get_model():
    model = ks.Sequential([
        ks.layers.Flatten(input_shape = (28, 28)),
        ks.layers.Dense(500, activation = 'relu'),
        ks.layers.Dense(400, activation = 'relu'),
        ks.layers.Dense(300, activation = 'relu'),
        ks.layers.Dense(200, activation = 'relu'),
        ks.layers.Dense(100, activation = 'relu'),
        ks.layers.Dense(10, activation = 'sigmoid')
    ])
    
    model.compile(optimizer = 'adam',
                 loss = 'categorical_crossentropy',
                 metrics = ['accuracy'])
    
    return model

In [ ]:
import time
# Timing the training process on CPU
start_time = time.time()

with tf.device('/cpu:0'):
    cpu_model = get_model()
    cpu_model.fit(X_train_scaled, y_train_categorical, epochs=5)

end_time = time.time()

# Output the time taken
print(f"Training on CPU took {end_time - start_time:.2f} seconds")

In [ ]:
start_time = time.time()

with tf.device('/gpu:0'):
    gpu_model = get_model()
    gpu_model.fit(X_train_scaled, y_train_categorical, epochs = 5)
    
end_time = time.time()

# Output of the time taken
print(f'Training on GPU took {end_time - start_time:.2f} seconds')

In [ ]:
# Model prediction
model.predict(X_test_scaled)[0]

In [ ]:
np.argmax(model.predict(X_test_scaled)[0])

In [ ]:
classes[np.argmax(model.predict(X_test_scaled)[0])]

In [ ]:
## Knowing the accuracy of the model

model.evaluate(X_test_scaled, y_test_categorical)